<a href="https://colab.research.google.com/github/SovetovAleksey/PyTorch/blob/main/HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
- Обернуть его в Dataloader
- Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
- Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [ ]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset
import torchmetrics

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
california_housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target,
                                                    test_size=0.25, random_state=13)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
X_train[0]

tensor([-0.1815,  0.5791, -0.3400, -0.0045, -0.0654, -0.0871, -0.6718,  0.6756])

In [ ]:
class CaliforniaDataset(torch.utils.data.Dataset):

    def __init__(self, *init_datasets):
        self._base_datasets = init_datasets

    def __len__(self):
        return len(self._base_datasets[0])

    def __getitem__(self, idx):
        return tuple(dataset[idx] for dataset in self._base_datasets)

In [ ]:
class CaliforniaNet(nn.Module):
    def __init__(self):
        super(CaliforniaNet, self).__init__()
        self.s1 = nn.Sequential(
            nn.Linear(in_features=8, out_features=500, bias=True),
            nn.Dropout(0.2),
            nn.BatchNorm1d(500),
            nn.LeakyReLU())
        self.s2 = nn.Sequential(
            nn.Linear(in_features=500, out_features=400, bias=True),
            nn.Dropout(0.2),
            nn.BatchNorm1d(400),
            nn.LeakyReLU())
        self.s3 = nn.Sequential(
            nn.Linear(in_features=400, out_features=100, bias=True),
            nn.Dropout(0.1),
            nn.BatchNorm1d(100),
            nn.LeakyReLU())
        self.s4 = nn.Sequential(
            nn.Linear(in_features=100, out_features=1, bias=True),
            nn.ReLU())

    def forward(self, x):
        x = self.s1(x)
        x = self.s2(x)
        x = self.s3(x)
        x = self.s4(x)

        return x[:, 0]

In [ ]:
train_dataset = CaliforniaDataset(X_train, y_train)
test_dataset = CaliforniaDataset(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
def train_val_net(net, optimizer):

    criterion = torch.nn.MSELoss(reduction='sum')
    train_r2 = torchmetrics.R2Score()
    test_r2 = torchmetrics.R2Score()

    net.train()

    for epoch in range(20):

        running_loss = 0.0
        running_items = 0.0

        for i, (inputs, labels) in enumerate(train_loader):

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            train_r2(outputs.squeeze(), labels)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            running_items += len(labels)

            if i % 150 == 0:    # печатаем каждые 80 mini-batches
                net.eval()
                print(f'Epoch: {epoch + 1} | loss: {running_loss / running_items:.5f} | r2_train: {train_r2.compute():.5f} | ',
                     end='')

                running_loss = 0.0
                running_items = 0.0
                train_r2.reset()

                for i, (test_inputs, test_labels) in enumerate(test_loader):
                    test_outputs = net(test_inputs)
                    test_r2(test_outputs.squeeze(), test_labels)

                print(f'r2_val: {test_r2.compute():.5f}')
                test_r2.reset()
                net.train()

    print('Training is finished!')

In [ ]:
LR = 0.001

In [ ]:
net = CaliforniaNet()
optimiser = torch.optim.Adam(net.parameters(), lr=LR)
train_val_net(net, optimiser)

Epoch: 1 | loss: 5.26765 | r2_train: -2.60437 | r2_val: -3.17550
Epoch: 1 | loss: 0.88187 | r2_train: 0.33865 | r2_val: 0.67392
Epoch: 2 | loss: 0.39426 | r2_train: 0.62351 | r2_val: 0.68656
Epoch: 2 | loss: 0.43909 | r2_train: 0.66856 | r2_val: 0.66668
Epoch: 3 | loss: 0.34769 | r2_train: 0.68237 | r2_val: 0.63897
Epoch: 3 | loss: 0.40302 | r2_train: 0.69626 | r2_val: 0.71634
Epoch: 4 | loss: 0.67093 | r2_train: 0.70802 | r2_val: 0.73693
Epoch: 4 | loss: 0.38525 | r2_train: 0.70433 | r2_val: 0.67049
Epoch: 5 | loss: 0.39372 | r2_train: 0.71012 | r2_val: 0.71836
Epoch: 5 | loss: 0.36432 | r2_train: 0.72706 | r2_val: 0.74615
Epoch: 6 | loss: 0.31068 | r2_train: 0.69810 | r2_val: 0.66227
Epoch: 6 | loss: 0.36523 | r2_train: 0.72213 | r2_val: 0.67308
Epoch: 7 | loss: 0.37293 | r2_train: 0.73104 | r2_val: 0.62090
Epoch: 7 | loss: 0.35855 | r2_train: 0.73018 | r2_val: 0.51123
Epoch: 8 | loss: 0.42599 | r2_train: 0.73155 | r2_val: 0.71297
Epoch: 8 | loss: 0.36504 | r2_train: 0.72497 | r2_val

In [ ]:
net = CaliforniaNet()
optimiser = RMSprop(net.parameters(), lr=LR)
train_val_net(net, optimiser)

Epoch: 1 | loss: 5.86754 | r2_train: -2.55818 | r2_val: -1.76302
Epoch: 1 | loss: 0.65819 | r2_train: 0.50920 | r2_val: 0.64462
Epoch: 2 | loss: 0.36364 | r2_train: 0.66382 | r2_val: 0.71089
Epoch: 2 | loss: 0.44424 | r2_train: 0.66060 | r2_val: 0.52385
Epoch: 3 | loss: 0.61003 | r2_train: 0.69679 | r2_val: 0.72403
Epoch: 3 | loss: 0.41292 | r2_train: 0.68747 | r2_val: 0.64482
Epoch: 4 | loss: 0.32958 | r2_train: 0.71454 | r2_val: 0.72214
Epoch: 4 | loss: 0.37891 | r2_train: 0.71173 | r2_val: 0.55414
Epoch: 5 | loss: 0.30745 | r2_train: 0.70943 | r2_val: 0.73454
Epoch: 5 | loss: 0.37702 | r2_train: 0.72016 | r2_val: 0.68378
Epoch: 6 | loss: 0.27503 | r2_train: 0.72113 | r2_val: 0.75393
Epoch: 6 | loss: 0.37143 | r2_train: 0.72192 | r2_val: 0.76127
Epoch: 7 | loss: 0.47732 | r2_train: 0.72491 | r2_val: 0.72556
Epoch: 7 | loss: 0.35755 | r2_train: 0.73282 | r2_val: 0.52292
Epoch: 8 | loss: 0.28500 | r2_train: 0.73174 | r2_val: 0.75471
Epoch: 8 | loss: 0.35062 | r2_train: 0.73524 | r2_val

In [ ]:
net = CaliforniaNet()
optimiser = SGD(net.parameters(), lr=LR)
train_val_net(net, optimiser)

Epoch: 1 | loss: 6.25981 | r2_train: -2.87747 | r2_val: -2.37986
Epoch: 1 | loss: 0.81133 | r2_train: 0.37999 | r2_val: 0.63633
Epoch: 2 | loss: 0.55399 | r2_train: 0.57841 | r2_val: 0.62188
Epoch: 2 | loss: 0.50839 | r2_train: 0.61591 | r2_val: 0.63411
Epoch: 3 | loss: 0.44242 | r2_train: 0.61591 | r2_val: 0.57334
Epoch: 3 | loss: 0.47679 | r2_train: 0.63522 | r2_val: 0.63295
Epoch: 4 | loss: 0.28293 | r2_train: 0.65587 | r2_val: 0.64729
Epoch: 4 | loss: 0.43806 | r2_train: 0.66955 | r2_val: 0.51230
Epoch: 5 | loss: 0.41008 | r2_train: 0.66990 | r2_val: 0.63512
Epoch: 5 | loss: 0.43484 | r2_train: 0.67274 | r2_val: 0.67872
Epoch: 6 | loss: 0.26811 | r2_train: 0.65326 | r2_val: 0.67263
Epoch: 6 | loss: 0.41668 | r2_train: 0.68454 | r2_val: 0.65187
Epoch: 7 | loss: 0.46506 | r2_train: 0.69902 | r2_val: 0.62359
Epoch: 7 | loss: 0.39873 | r2_train: 0.69764 | r2_val: 0.69279
Epoch: 8 | loss: 0.40521 | r2_train: 0.69111 | r2_val: 0.67948
Epoch: 8 | loss: 0.40378 | r2_train: 0.69598 | r2_val